In [1]:
!pip install pyspark

    100% |████████████████████████████████| 188.2MB 4.5kB/s ta 0:00:011   11% |███▊                            | 22.1MB 11.0MB/s eta 0:00:16    14% |████▌                           | 26.8MB 12.9MB/s eta 0:00:13    30% |█████████▋                      | 56.9MB 3.7MB/s eta 0:00:36    30% |█████████▉                      | 57.7MB 13.6MB/s eta 0:00:10    31% |██████████                      | 58.4MB 14.8MB/s eta 0:00:09    37% |████████████                    | 70.2MB 10.9MB/s eta 0:00:11    41% |█████████████▏                  | 77.6MB 11.8MB/s eta 0:00:10    41% |█████████████▎                  | 78.1MB 12.2MB/s eta 0:00:10    48% |███████████████▋                | 91.8MB 19.8MB/s eta 0:00:05    51% |████████████████▍               | 96.1MB 21.3MB/s eta 0:00:05    53% |█████████████████               | 99.7MB 13.3MB/s eta 0:00:07    53% |█████████████████               | 100.2MB 15.7MB/s eta 0:00:06    56% |██████████████████              | 105.8MB 14.8MB/s eta 0:00:06    57% |██████████

In [2]:
import sys
sys.path.insert(0, "../")
from garrens_utils import *

In [3]:
spark

In [7]:
from pyspark.ml import Pipeline, PipelineModel
from sparknlp.annotator import *
from sparknlp.base import DocumentAssembler, Finisher
from pyspark.ml.feature import StopWordsRemover

In [4]:
reviews = spark.read.json("../data/input/reviews_Clothing_Shoes_and_Jewelry_5.json.gz")
reviews.cache().count() # FAST is GOOD - ~115MB cached vs 146MB raw vs 54MB GZipped
reviews.createOrReplaceTempView("reviews")

### Build Lemma dictionary

In [47]:
# Unused due to strange results
# from nltk.corpus import wordnet as wn
# import nltk
# with open("../data/input/lemma_dictionary.txt", "w") as f:
#     for lemma in wn.all_lemma_names():
#         f.write(lemma + " -> " + ",".join(set(syn.name().split('.')[0] or syn.name().split('.')[1] for syn in wn.synsets(lemma))) + "\n")

In [8]:
document_assembler = DocumentAssembler() \
            .setInputCol("reviewText")

sentence_detector = SentenceDetector() \
            .setInputCols(["document"]) \
            .setOutputCol("sentence")
    
tokenizer = Tokenizer() \
            .setInputCols(["sentence"]) \
            .setOutputCol("token")
        
normalizer = Normalizer() \
            .setInputCols(["token"]) \
            .setOutputCol("normal")
        
spell_checker = NorvigSweetingApproach() \
            .setInputCols(["normal"]) \
            .setOutputCol("spell")
    
finisher = Finisher() \
    .setInputCols(["spell"]) \
    .setOutputCols(["spelt_words"])

In [9]:
pipeline = Pipeline(stages=[document_assembler, sentence_detector, tokenizer, normalizer, spell_checker, finisher])
model = pipeline.fit(reviews)
result = model.transform(reviews)

In [ ]:
# *** Skip processing on this notebook due to slowness ***
# result_split = result.withColumn("words", F.expr("split(spelt_words,'@') AS words"))

# remover = StopWordsRemover()\
#             .setInputCol("words")\
#             .setOutputCol("filtered")\
#             .setStopWords(StopWordsRemover.loadDefaultStopWords("english"))

# result_split_no_stop = remover.transform(result_split)

# result_summary = result_split_no_stop.selectExpr("*", "explode(filtered) AS word")\
#     .groupBy("word", "overall")\
#     .agg(
#         F.countDistinct("reviewerID").alias("unique_reviewers"), 
#         F.countDistinct("asin").alias("unique_products"), 
#         F.count(F.lit(1)).alias("times_seen"),
#         F.sum(F.expr("helpful[0]")).alias("is_helpful"),
#         F.sum(F.expr("helpful[1]-helpful[0]")).alias("not_helpful")
#     )

# result_summary.repartition(8).write.mode("overwrite").parquet("../data/output/word_freq_summary")

In [18]:
result_summary = spark.read.parquet("../data/output/word_freq_summary")
result_summary.cache().count()
result_summary.createOrReplaceTempView("result_summary")

In [19]:
word_freq = spark.sql("""SELECT word, 
SUM(is_helpful) AS total_helpful, 
SUM(not_helpful) AS total_unhelpful, 
SUM(times_seen) AS total_occurrences
FROM result_summary
GROUP BY word""")
word_freq.cache().count()
word_freq.createOrReplaceTempView("word_frequency")

In [ ]:
# *** Skip processing on this notebook due to slowness ***
# wf_props = spark.sql("""SELECT word,
# total_helpful,
# total_unhelpful,
# SUM(coalesce(rating_1_helpful_prop,0.0)) AS rating_1_helpful_prop,
# SUM(coalesce(rating_2_helpful_prop,0.0)) AS rating_2_helpful_prop,
# SUM(coalesce(rating_3_helpful_prop,0.0)) AS rating_3_helpful_prop,
# SUM(coalesce(rating_4_helpful_prop,0.0)) AS rating_4_helpful_prop,
# SUM(coalesce(rating_5_helpful_prop,0.0)) AS rating_5_helpful_prop,
# SUM(coalesce(rating_1_unhelpful_prop,0.0)) AS rating_1_unhelpful_prop,
# SUM(coalesce(rating_2_unhelpful_prop,0.0)) AS rating_2_unhelpful_prop,
# SUM(coalesce(rating_3_unhelpful_prop,0.0)) AS rating_3_unhelpful_prop,
# SUM(coalesce(rating_4_unhelpful_prop,0.0)) AS rating_4_unhelpful_prop,
# SUM(coalesce(rating_5_unhelpful_prop,0.0)) AS rating_5_unhelpful_prop,
# SUM(coalesce(rating_1_helpfulness_ratio,0.0)) AS rating_1_helpfulness_ratio,
# SUM(coalesce(rating_2_helpfulness_ratio,0.0)) AS rating_2_helpfulness_ratio,
# SUM(coalesce(rating_3_helpfulness_ratio,0.0)) AS rating_3_helpfulness_ratio,
# SUM(coalesce(rating_4_helpfulness_ratio,0.0)) AS rating_4_helpfulness_ratio,
# SUM(coalesce(rating_5_helpfulness_ratio,0.0)) AS rating_5_helpfulness_ratio
# FROM
# (
# 	SELECT word, 
# 	CASE WHEN overall = 1.0 THEN is_helpful/total_helpful END rating_1_helpful_prop,
# 	CASE WHEN overall = 2.0 THEN is_helpful/total_helpful END rating_2_helpful_prop,
# 	CASE WHEN overall = 3.0 THEN is_helpful/total_helpful END rating_3_helpful_prop,
# 	CASE WHEN overall = 4.0 THEN is_helpful/total_helpful END rating_4_helpful_prop,
# 	CASE WHEN overall = 5.0 THEN is_helpful/total_helpful END rating_5_helpful_prop,
# 	CASE WHEN overall = 1.0 THEN not_helpful/total_unhelpful END rating_1_unhelpful_prop,
# 	CASE WHEN overall = 2.0 THEN not_helpful/total_unhelpful END rating_2_unhelpful_prop,
# 	CASE WHEN overall = 3.0 THEN not_helpful/total_unhelpful END rating_3_unhelpful_prop,
# 	CASE WHEN overall = 4.0 THEN not_helpful/total_unhelpful END rating_4_unhelpful_prop,
# 	CASE WHEN overall = 5.0 THEN not_helpful/total_unhelpful END rating_5_unhelpful_prop,
# 	CASE WHEN overall = 1.0 THEN is_helpful/not_helpful END rating_1_helpfulness_ratio,
# 	CASE WHEN overall = 2.0 THEN is_helpful/not_helpful END rating_2_helpfulness_ratio,
# 	CASE WHEN overall = 3.0 THEN is_helpful/not_helpful END rating_3_helpfulness_ratio,
# 	CASE WHEN overall = 4.0 THEN is_helpful/not_helpful END rating_4_helpfulness_ratio,
# 	CASE WHEN overall = 5.0 THEN is_helpful/not_helpful END rating_5_helpfulness_ratio,
# 	total_helpful,
# 	total_unhelpful
# 	FROM result_summary rs
# 	JOIN word_frequency wf USING (word)
# 	WHERE is_helpful > 0
# ) word_props
# GROUP BY word, total_helpful, total_unhelpful""")

# wf_props.repartition(8).write.mode("overwrite").parquet("../data/output/word_frequency_proportions")

In [33]:
wf_props = spark.read.parquet("../data/output/word_frequency_proportions")
wf_props.cache().count()
wf_props.createOrReplaceTempView("wf_props")

### Most helpful negative review words

In [35]:
%%sql_display
SELECT *
FROM
(
    SELECT *
    FROM wf_props
    ORDER BY total_helpful DESC
    LIMIT 1000
) top_1000_helpful_ratings
ORDER BY rating_1_helpful_prop DESC
LIMIT 25

,word,total_helpful,total_unhelpful,rating_1_helpful_prop,rating_2_helpful_prop,rating_3_helpful_prop,rating_4_helpful_prop,rating_5_helpful_prop,rating_1_unhelpful_prop,rating_2_unhelpful_prop,rating_3_unhelpful_prop,rating_4_unhelpful_prop,rating_5_unhelpful_prop,rating_1_helpfulness_ratio,rating_2_helpfulness_ratio,rating_3_helpfulness_ratio,rating_4_helpfulness_ratio,rating_5_helpfulness_ratio
0,returning,4894,1185,0.326522,0.144258,0.250102,0.124029,0.155088,0.381435,0.175527,0.254008,0.108017,0.081013,3.535398,3.394231,4.066445,4.742188,7.906250
1,label,4105,492,0.263825,0.046529,0.044336,0.302071,0.343240,0.402439,0.083333,0.069106,0.215447,0.229675,5.469697,4.658537,5.352941,11.698113,12.469027
2,return,19610,3679,0.238654,0.178276,0.178480,0.125803,0.278786,0.359609,0.217994,0.196521,0.102473,0.123403,3.537415,4.359102,4.840941,6.543767,12.041850
3,seller,10461,1635,0.215180,0.176083,0.100468,0.108594,0.399675,0.286239,0.179817,0.097859,0.113150,0.322936,4.809829,6.265306,6.568750,6.140541,7.918561
4,returned,8482,2093,0.209149,0.219995,0.191818,0.118250,0.260788,0.308648,0.310559,0.182035,0.096990,0.101768,2.746130,2.870769,4.270341,4.940887,10.384977
5,poor,5240,843,0.207824,0.206107,0.142939,0.085687,0.357443,0.342823,0.334520,0.139976,0.067616,0.115065,3.768166,3.829787,6.347458,7.877193,19.309278
6,apart,5687,825,0.195358,0.128890,0.154563,0.120978,0.400211,0.298182,0.181818,0.134545,0.099394,0.286061,4.516260,4.886667,7.918919,8.390244,9.644068
7,money,17668,3093,0.182760,0.082975,0.105162,0.172119,0.456984,0.352409,0.124151,0.120272,0.119948,0.283220,2.962385,3.817708,4.994624,8.196765,9.216895
8,bras,16874,1416,0.175714,0.032950,0.082553,0.202975,0.505808,0.238701,0.081215,0.132062,0.138418,0.409605,8.772189,4.834783,7.449198,17.474490,14.715517
9,th,4248,603,0.175377,0.019303,0.076271,0.165960,0.563089,0.285240,0.077944,0.132670,0.131012,0.373134,4.331395,1.744681,4.050000,8.924051,10.631111


In [36]:
%%sql_display
SELECT *
FROM
(
    SELECT *
    FROM wf_props
    ORDER BY total_helpful DESC
    LIMIT 1000
) top_1000_helpful_ratings
ORDER BY rating_5_helpful_prop DESC
LIMIT 25

,word,total_helpful,total_unhelpful,rating_1_helpful_prop,rating_2_helpful_prop,rating_3_helpful_prop,rating_4_helpful_prop,rating_5_helpful_prop,rating_1_unhelpful_prop,rating_2_unhelpful_prop,rating_3_unhelpful_prop,rating_4_unhelpful_prop,rating_5_unhelpful_prop,rating_1_helpfulness_ratio,rating_2_helpfulness_ratio,rating_3_helpfulness_ratio,rating_4_helpfulness_ratio,rating_5_helpfulness_ratio
0,trifold,7194,140,0.000000,0.000000,0.000278,0.000139,0.999583,0.000000,0.000000,0.014286,0.000000,0.978571,0.000000,0.000000,1.000000,0.000000,52.489051
1,spaces,9654,192,0.000518,0.004972,0.000725,0.003936,0.989849,0.015625,0.041667,0.010417,0.057292,0.875000,1.666667,6.000000,3.500000,3.454545,56.880952
2,cincher,6092,792,0.003283,0.000657,0.018056,0.010506,0.967498,0.016414,0.003788,0.058081,0.021465,0.900253,1.538462,1.333333,2.391304,3.764706,8.266480
3,wallets,4488,133,0.003565,0.003342,0.010918,0.021836,0.960339,0.022556,0.082707,0.112782,0.037594,0.744361,5.333333,1.363636,3.266667,19.600000,43.535354
4,scared,4098,369,0.001952,0.001464,0.006101,0.033431,0.957052,0.130081,0.000000,0.018970,0.238482,0.612466,0.166667,0.000000,3.571429,1.556818,17.353982
5,vest,7522,968,0.012896,0.005982,0.010503,0.027386,0.943233,0.011364,0.022727,0.021694,0.035124,0.909091,8.818182,2.045455,3.761905,6.058824,8.062500
6,horse,6677,395,0.012880,0.005691,0.002396,0.037592,0.941441,0.055696,0.010127,0.012658,0.025316,0.896203,3.909091,9.500000,3.200000,25.100000,17.757062
7,bifold,6195,123,0.000161,0.000000,0.000323,0.062147,0.937369,0.000000,0.000000,0.016260,0.146341,0.837398,0.000000,0.000000,1.000000,21.388889,56.378641
8,cz,3991,263,0.003508,0.007016,0.027061,0.030068,0.932348,0.045627,0.083650,0.041825,0.068441,0.760456,1.166667,1.272727,9.818182,6.666667,18.605000
9,mouth,4247,277,0.011538,0.024723,0.003532,0.028726,0.931481,0.079422,0.079422,0.007220,0.032491,0.801444,2.227273,4.772727,7.500000,13.555556,17.819820


In [39]:
%%sql_display
SELECT *
FROM wf_props
WHERE word = 'amazing'

,word,total_helpful,total_unhelpful,rating_1_helpful_prop,rating_2_helpful_prop,rating_3_helpful_prop,rating_4_helpful_prop,rating_5_helpful_prop,rating_1_unhelpful_prop,rating_2_unhelpful_prop,rating_3_unhelpful_prop,rating_4_unhelpful_prop,rating_5_unhelpful_prop,rating_1_helpfulness_ratio,rating_2_helpfulness_ratio,rating_3_helpfulness_ratio,rating_4_helpfulness_ratio,rating_5_helpfulness_ratio
0,amazing,8557,1043,0.024775,0.015192,0.018932,0.090335,0.850765,0.064238,0.028763,0.060403,0.1093,0.737296,3.164179,4.333333,2.571429,6.780702,9.46684


## Pick a word, any word

In [46]:
selected_word = 'awesome'
sw_result = wf_props.filter("word = '{sw}'".format(sw=selected_word)).first()

In [55]:
sw_result.asDict()

{'rating_1_helpful_prop': 0.02196042548324374,
 'rating_1_helpfulness_ratio': 3.2,
 'rating_1_unhelpful_prop': 0.05258545135845749,
 'rating_2_helpful_prop': 0.06816882077090243,
 'rating_2_helpfulness_ratio': 8.514285714285714,
 'rating_2_unhelpful_prop': 0.06134969325153374,
 'rating_3_helpful_prop': 0.058789889054100423,
 'rating_3_helpfulness_ratio': 5.410526315789474,
 'rating_3_unhelpful_prop': 0.08326029798422437,
 'rating_4_helpful_prop': 0.10053757291547524,
 'rating_4_helpfulness_ratio': 6.8671875,
 'rating_4_unhelpful_prop': 0.11218229623137599,
 'rating_5_helpful_prop': 0.7505432917762782,
 'rating_5_helpfulness_ratio': 8.32741116751269,
 'rating_5_unhelpful_prop': 0.6906222611744084,
 'total_helpful': 8743,
 'total_unhelpful': 1141,
 'word': 'awesome'}

In [74]:
print("Word: {0}".format(sw_result.word))
print("Total Reviews (1-5) marked Helpful: {0}".format(sw_result.total_helpful))
print("Total Reviews (1-5) marked Unhelpful: {0}".format(sw_result.total_unhelpful))
print("Rating 1 helpfulness ratio (closer to 0 means less helpful): {0}".format(sw_result.rating_1_helpfulness_ratio))
print("Rating 5 helpfulness ratio (closer to 0 means less helpful): {0}".format(sw_result.rating_5_helpfulness_ratio))
print("Rating 1 % of reviews rated helpful: {:.0%}".format(sw_result.rating_1_helpful_prop))
print("% of reviews with Rating of 1.0 rated helpful: {:.0%}".format(sw_result.rating_1_helpful_prop))
print("% of reviews with Rating of 5.0 rated helpful: {:.0%}".format(sw_result.rating_5_helpful_prop))
is_positive_word = None
if sw_result.rating_5_helpful_prop > sw_result.rating_1_helpful_prop and \
    sw_result.rating_5_helpfulness_ratio > sw_result.rating_1_helpfulness_ratio:
    is_positive_word = True
else:
    is_positive_word = False
print("Likely word interpretation: {0}".format("Positive" if is_positive_word else "Negative"))

Word: awesome
Total Reviews (1-5) marked Helpful: 8743
Total Reviews (1-5) marked Unhelpful: 1141
Rating 1 helpfulness ratio (closer to 0 means less helpful): 3.2
Rating 5 helpfulness ratio (closer to 0 means less helpful): 8.32741116751269
Rating 1 % of reviews rated helpful: 2%
% of reviews with Rating of 1.0 rated helpful: 2%
% of reviews with Rating of 5.0 rated helpful: 75%
Likely word interpretation: Positive


# TODO: Use review text to predict review rating - exercise for the reader :D

https://spark.apache.org/docs/latest/ml-guide.html